# Grounnding with Web Search Comparison

----



# Setup

Let's define our kernel for this example.

In [1]:
import asyncio
from typing import Annotated
from dotenv import load_dotenv
import json
import os
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.filters import FilterTypes, FunctionInvocationContext
from collections.abc import Awaitable, Callable
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.agents import AzureResponsesAgent, ResponsesAgentThread
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import AzureResponsesAgent
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents import AuthorRole
from semantic_kernel.contents.streaming_chat_message_content import StreamingChatMessageContent
from azure.ai.agents.models import OpenApiAnonymousAuthDetails, OpenApiTool, CodeInterpreterTool
from azure.ai.agents.models import BingGroundingTool, MessageRole
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,
)

from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.contents import (
    AnnotationContent,
    ChatMessageContent,
    FunctionCallContent,
    FunctionResultContent,
)
from langchain.prompts import PromptTemplate
import os
import pytz
from datetime import datetime
from IPython.display import Markdown, display

import httpx
import scrapy
from urllib.parse import urljoin
from openai import AsyncAzureOpenAI
    
load_dotenv(override=True)


current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

In [2]:
bing_api_key = os.getenv("BING_API_KEY")
bing_custom_config_id = os.getenv("BING_CUSTOM_CONFIG_ID")
bing_max_result = int(os.getenv("BING_MAX_RESULT", "10"))

def bing_search_api(query: str) -> list[dict]:
    """
    Perform a Bing search using the Bing Search API and return the results.
    
    Args:
        query (str): The search query.
        
    Returns:
        list[dict]: A list of search results, each containing 'link' and 'snippet'.
    """
    import requests

    if not bing_api_key:
        print("Bing API key is not set. Please set the BING_API_KEY environment variable.")
        return []
          
    
    if bing_custom_config_id:
        url = "https://api.bing.microsoft.com/v7.0/custom/search"
    else:
        url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {
        "Ocp-Apim-Subscription-Key": bing_api_key
    }
    params = {
        "q": query + " -filetype:pdf",
        "count": bing_max_result,
        "mkt": "ko-KR",
        "responseFilter": "Webpages",
    }
    if bing_custom_config_id:
        params.update({
            "customconfig": bing_custom_config_id,
        })

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        results = response.json()
        
        # Convert Bing format to match Google format for consistency
        formatted_results = []
        if "webPages" in results and "value" in results["webPages"]:
            for item in results["webPages"]["value"]:
                formatted_results.append({
                    "link": item.get("url"),
                    "name": item.get("name"),
                    "snippet": item.get("snippet"),
                    "displayLink": item.get("displayUrl")
                })
        
        return formatted_results
    except requests.RequestException as e:
        print(f"Bing search API error: {str(e)}")
        return []
    
async def fetch_data_from_url(url: str, snippet: str) -> str:
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    try:
        async with httpx.AsyncClient(timeout=10, follow_redirects=True) as client:
            try:
                response = await client.get(url, headers=headers)
                response.raise_for_status()
            except httpx.HTTPStatusError as e:
                if e.response.status_code == 302 and "location" in e.response.headers:
                    redirect_url = e.response.headers["location"]
                    if not redirect_url.startswith("http"):
                        redirect_url = urljoin(url, redirect_url)
                    try:
                        response = await client.get(redirect_url, headers=headers)
                        response.raise_for_status()
                    except Exception as e2:
                        print(f"Redirect request failed: {e2}")
                        return f"{snippet} "
                else:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
            except httpx.HTTPError as e:
                print(f"Request failed: {e}")
                return f"{snippet} "
            
            selector = scrapy.Selector(text=response.text)
            
            paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
            
            filtered_paragraphs = []
            seen_content = set()
            for p in paragraphs:
                if len(p) < 5:
                    continue
                if p in seen_content:
                    continue
                seen_content.add(p)
                filtered_paragraphs.append(p)
            
            text = "\n".join(filtered_paragraphs)
            
            if not text:
                content_texts = [t.strip() for t in selector.css(
                    'article::text, article *::text, .content::text, .content *::text, '
                    'main::text, main *::text'
                ).getall() if t.strip()]
                
                if content_texts:
                    text = "\n".join(content_texts)
            
            snippet_text = f"{snippet}: {text}"
            
            return snippet_text
            
    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")
        return f"{snippet} [Error: {str(e)}]"

In [3]:
bing_search_api("최근 SNS에서 자주 사용되는 해시태그는?")

[{'link': 'https://ddobong100.tistory.com/35',
  'name': '2025년 SNS에서 인기 있는 해시태그 분석! 트렌드를 읽는 꿀팁',
  'snippet': 'SNS를 활용하는 데 있어 해시태그는 콘텐츠의 노출과 도달률을 결정짓는 중요한 요소 죠. 해시태그만 잘 활용해도 내 게시물이 더 많은 사람들에게 도달할 수 있어요! 오늘은 2025년 SNS에서 인기 있는 해시태그 를 분석하고, 이를 활용해 콘텐츠를 효과적으로 운영하는 방법을 소개할게요.',
  'displayLink': 'https://ddobong100.tistory.com/35'},
 {'link': 'https://snskeyboard.com/hashtag/',
  'name': '#️⃣인스타그램 인기 해시태그 모음 ― SNS Keyboard',
  'snippet': "이 페이지에서 저희는 인스타그램에서 자주 사용되는 해시태그 (Hash Tag, #)에 대해 정리해 보았어요. 해시태그란 인스타그램, 트위터와 같은 SNS에서 사용되는 태그 기호에요. '#' 뒤에 어떤 단어를 입력하면 그 단어에 대한 포스팅 글을 한데 모아서 볼 수 있죠.",
  'displayLink': 'https://snskeyboard.com/hashtag'},
 {'link': 'https://hyddy.tistory.com/122',
  'name': '2025년 인스타그램 인기 해시태그 TOP 10',
  'snippet': '2025년 인스타그램 인기 해시태그 TOP 10여러분, 인스타그램에서 더 많은 관심과 참여를 받고 싶으신가요? 그렇다면 2025년 최신 인기 해시태그를 확인해보세요!안녕하세요, 여러분!',
  'displayLink': 'https://hyddy.tistory.com/122'},
 {'link': 'https://8v8kwn.tistory.com/1025',
  'name': '인스타그램 해시태그 최신 목록과 사용 팁 - Tor',
  'snippet': '

## 🧪 Case 1. Bing search API -> Scraping -> LLM generation (deprecated) 
---

In [12]:
USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"

search_results = bing_search_api(USER_INPUT)

all_contexts = []

    
if search_results:
    url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
    
    tasks = [asyncio.create_task(fetch_data_from_url(url, snippet)) 
                    for url, snippet in url_snippet_tuples]


    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    contexts = []    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Error processing URL {url_snippet_tuples[i][0]}: {str(result)}")
            contexts.append(f"{url_snippet_tuples[i][1]} [Processing Error]")
        else:
            contexts.append(result)

    contexts_text = "\n\n".join(contexts)


SYSTEM_PROMPT = """    
    너는 웹에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

    * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
    * 아래 제공하는 검색엔진에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
    * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
    * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
    * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
    * 출력언어 설정에 따라 답변해주세요. 

"""

USER_PROMPT_TEMPLATE = """    
    오늘 날짜는 {current_date}이며, 가능한 한 최신의 데이터를 기반으로 사용자 질문에 답변해주세요. 
   
    사용자 질문:
    {query}
    검색엔진에서 검색된 컨텍스트: 
    {contexts}
    검색된 참조링크:
    {url_citation}
    출력언어:
    {locale}

"""

USER_PROMPT = PromptTemplate(
    template=USER_PROMPT_TEMPLATE,
    input_variables=["query","current_date","contexts"],
    optional_variables=["url_citation","locale"]

)

answer_messages = [{"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT.format(
                        current_date=current_date,
                        contexts=contexts,
                        query=USER_INPUT,
                        url_citation=json.dumps(url_snippet_tuples, ensure_ascii=False),
                        locale="ko-KR"
                    )}]


client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

stream = False

response = await client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                messages=answer_messages,
                max_tokens=2000,
                temperature=0.7,
                stream=stream
            )
            
if stream:
    async for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            #print(chunk.choices[0].delta.content, end="", flush=True)
            display(Markdown(chunk.choices[0].delta.content), clear=True)

else:
    message_content = response.choices[0].message.content
    display(Markdown(message_content))



검색된 결과 수: 10


서울 도심에서 가장 최근에 오픈한 핫플레이스 중 하나는 **구리 한강시민공원 리버뷰 전망대**입니다. 이 전망대는 2025년에 새롭게 리모델링되어 한강의 아름다운 경관을 감상할 수 있는 최적의 장소로 인기를 끌고 있습니다. 특히 SNS에서 많은 사랑을 받고 있으며, 이곳에서 찍은 사진들이 많은 인기를 얻고 있습니다. 📸🌅

또한, **가평 모닝뷰 카페**도 2025년 초에 오픈한 핫플레이스로, 통유리창을 통해 북한강을 내려다보며 아침 햇살을 즐길 수 있는 특별한 경험을 제공합니다. 이 카페는 특히 '클라우드 라떼'로 유명하며, 인스타그램에서도 핫한 장소로 떠오르고 있습니다. 🥤✨

서울 도심 내에서의 새로운 핫플레이스를 찾고 계신다면, 성수동과 익선동 지역도 추천드립니다. 이 두 지역은 현대적 감각의 카페와 독특한 상점들로 가득 차 있어, 새로운 문화와 경험을 찾는 분들에게 최적의 장소입니다.

더 많은 정보는 아래의 링크를 참고하세요:
- [2025년에 새로 생긴 서울 외곽 핫플레이스는 어디인가요?](https://kiuas.tistory.com/239)
- [2025년 서울 핫플레이스 TOP 10](https://view1976.tistory.com/100) 

서울의 새로운 핫플레이스들을 방문하여 특별한 추억을 만들어보세요! 😊

## 🧪 Case 2. Semantic Kernel Connector (LLM -> bing search as Tool) 
---

In [6]:
USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"

thread: ChatHistoryAgentThread = None

webplugin = KernelPlugin.from_text_search_with_search(
        BingSearch(api_key=os.getenv("BING_API_KEY")),
        plugin_name="bing",
        description="Search the web for information.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=1,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            # KernelParameterMetadata(
            #     name="site",
            #     description="The site to search.",
            #     default_value="https://github.com/microsoft/semantic-kernel/tree/main/python",
            #     type="str",
            #     is_required=False,
            #     type_object=str,
            # ),
        ],
    )

execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="chat",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    function_choice_behavior=FunctionChoiceBehavior.Auto(auto_invoke=True),
)

PROMPT_TEMPLATE = """
    너는 웹에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

    * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
    * 오늘 날짜는 {current_date}이며, 가능한 한 최신의 데이터를 기반으로 답변해주세요.  
    * 검색엔진에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
    * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
    * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
    * 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[name](link)`)으로 포함해주세요.
    * 부정확하거나 예시 링크(예: https://example.com)는 절대 포함하지 말고 웹검색에서 조회한 사이트를 참조 링크로 사용해주세요.
    * 출력언어 설정에 따라 답변해주세요. 

    
  사용자질문: 
  {query}
  출력언어:
  {locale}

"""

SYSTEM_PROMPT = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["query","current_date"],
    optional_variables=["locale"]

)

agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions=SYSTEM_PROMPT.format(
            query=USER_INPUT,
            current_date=current_date,
            locale="ko-KR",
        ),
        plugins=[webplugin],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

# 4. Invoke the agent for a response
response = await agent.get_response(messages=USER_INPUT, thread=thread)
print(f"# {response.name}")
display(Markdown(response.message.content))
thread = response.thread


#######################################
# invoke_stream mode 
#######################################

# print(f"# User: '{USER_INPUT}'")
# first_chunk = True
# # 4. Invoke the agent for the current message and print the response
# async for response in agent.invoke_stream(messages=USER_INPUT, thread=thread):
#     thread = response.thread
#     if first_chunk:
#         print(f"# {response.name}: ", end="", flush=True)
#         first_chunk = False
#     print(response.content, end="", flush=True)
# print()

await thread.delete() if thread else None

# Host


최근 서울 도심에서 새롭게 오픈한 핫플레이스 중 하나는 **'서울숲 라운지'**입니다. 이곳은 자연을 느끼며 편안하게 시간을 보낼 수 있는 공간으로, 서울숲 인근에 위치하고 있습니다. 다양한 음식과 음료를 제공하며, 특히 인스타그램 인기 사진 명소로 떠오르고 있습니다. 🌳✨

또한, **'신사동 가로수길의 핫플레이스'**도 주목할 만한 오픈 장소입니다. 여러 인기 카페와 패션 스토어가 함께하는 공간으로, 트렌디한 문화 공간으로 자리잡고 있습니다. 최근에는 ‘카페 파리크라상'에서 오는 브랜딩 이벤트도 진행되고 있어 방문객들의 관심을 끌고 있습니다.

이 외에도 다양한 신규 오픈 장소가 서울 곳곳에서 활발히 선보이고 있으며, 이러한 장소들은 소셜 미디어를 통해 더욱 많은 사랑을 받고 있습니다. 더 많은 핫플레이스에 대한 정보를 원하시면 이 [링크](https://www.yonhapnews.co.kr/society/20230702000100033.html)를 확인하세요. 

서울의 다양한 핫플레이스를 탐방해보세요! 🎉

## 🧪 Case 3. BingGrounding with Azure AI Agent (SK)
---

#### ❗​run "az login --use-device-code" command in your terminal to login to Azure CLI
https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively?view=azure-cli-latest

In [11]:
USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"
print(f"# User: '{USER_INPUT}'")
            
from azure.identity.aio import DefaultAzureCredential

BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
# BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 5))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
    
ai_agent_settings = AzureAIAgentSettings()

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        elif isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"{item}")

async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # 1. Enter your Bing Grounding Connection Name
        bing_connection = await client.connections.get(name=BING_GROUNDING_CONNECTION_NAME)
        conn_id = bing_connection.id

        # 2. Initialize agent bing tool and add the connection id
        bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=int(BING_GROUNDING_MAX_RESULTS))


        SYSTEM_PROMPT = """    
            너는 bing search tool에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

            * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
            * bing search tool에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
            * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
            * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
            * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
            * 출력언어 설정에 따라 답변해주세요. 

        """

        USER_PROMPT_TEMPLATE = """    
            오늘 날짜는 {current_date}이며, bing search tool에서 검색된 최근 결과를 중심으로 사용자 질문에 답변해주세요. 
        
            사용자 질문:
            {query}
            출력언어:
            {locale}

        """

        USER_PROMPT = PromptTemplate(
            template=USER_PROMPT_TEMPLATE,
            input_variables=["query","current_date"],
            optional_variables=["locale"]

        )


        # 3. Create an agent with Bing grounding on the Azure AI agent service
        agent_definition = await client.agents.create_agent(
            name="SKBingGroundingAgent",
            instructions=SYSTEM_PROMPT,
            model=AzureAIAgentSettings().model_deployment_name,
            tools=bing.definitions,
        )

        # 4. Create a Semantic Kernel agent for the Azure AI agent
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )

        # 5. Create a thread for the agent
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        
        
        current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

        try:
            # 6. Invoke the agent for the specified thread for response
            async for response in agent.invoke(
                messages=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    locale="ko-KR",
                ), thread=thread, on_intermediate_message=handle_intermediate_steps
            ):
                print(f"# {response.name}:")
                display(Markdown(response.message.content))
                thread = response.thread

                # 7. Show annotations
                if any(isinstance(item, AnnotationContent) for item in response.items):
                    for annotation in response.items:
                        if isinstance(annotation, AnnotationContent):
                            print(
                                f"Annotation :> {annotation.url}, source={annotation.quote}, with "
                                f"start_index={annotation.start_index} and end_index={annotation.end_index}"
                            )
        finally:
            # 8. Cleanup: Delete the thread and agent
            await thread.delete() if thread else None
            await client.agents.delete_agent(agent.id)



# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'


Function Call:> bing_grounding with arguments: {'requesturl': 'https://api.bing.microsoft.com/v7.0/search?q=서울 도심 새로 오픈한 핫플레이스 2025년 7월', 'response_metadata': "{'market': 'ko-KR', 'num_docs_retrieved': 3, 'num_docs_actually_used': 3}"}
# SKBingGroundingAgent:


최근 서울 도심에서 주목받는 핫플레이스 중에서는 아래 몇 가지가 눈에 띕니다:

1. **쿠키런 브레이브 이스케이프 팝업스토어**  
   - 위치: 성수동 (서울 성동구 성수이로18길)  
   - 운영 기간: 2025년 5월 28일 ~ 8월 28일  
   - 특징: 인기 게임 쿠키런 테마로 다양한 포토존과 굿즈를 제공하며, 체험형 콘텐츠로 즐길 거리가 많습니다【3:0†source】.

2. **디지몬 어드벤처 25주년 기념전**  
   - 위치: 홍대 AK플라자 4층 (서울 마포구 양화로 188)  
   - 운영 기간: 2025년 5월 1일 ~ 7월 9일  
   - 특징: 디지몬 팬들을 위한 전시로, 다양한 전시물과 한정 굿즈가 준비되어 있습니다【3:0†source】.

3. **불가리 팝업스토어**  
   - 위치: 강남 현대백화점 무역센터점 (서울 강남구 테헤란로 517)  
   - 운영 기간: 2025년 4월 1일 ~ 7월 31일  
   - 특징: 럭셔리 브랜드 불가리의 다양한 제품을 경험할 수 있는 공간으로, 쇼핑과 함께 고급스러움을 만끽할 수 있는 장소입니다【3:0†source】.

다양한 테마와 특별한 경험을 제공하는 서울의 이 새로운 공간들은 사진 찍기 좋은 포토존도 많아서 인스타그램 업로드용으로도 딱이에요! 📸✨

Annotation :> https://blog.naver.com/rentalcenter1/223908648557, source=【3:0†source】, with start_index=205 and end_index=217
Annotation :> https://blog.naver.com/rentalcenter1/223908648557, source=【3:0†source】, with start_index=371 and end_index=383
Annotation :> https://blog.naver.com/rentalcenter1/223908648557, source=【3:0†source】, with start_index=557 and end_index=569


## 🧪 Case 4. BingGrounding with Azure AI Agent (Vanila)
---

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from IPython.display import Markdown, display

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")

BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")


# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=BING_GROUNDING_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=int(BING_GROUNDING_MAX_RESULTS))

with project_client:
    

    SYSTEM_PROMPT = """    
        너는 bing search tool에서 검색한 정보를 바탕으로 사용자 질문에 대한 답변을 제공하는 챗봇이야. 다음의 규칙을 따라 답변해줘. 

        * 사실에 입각한 정확성, 명확성, 그리고 실행 가능한 인사이트 제공에 중점을 두어 작성해주세요.
        * bing search tool에서 검색한 결과를 컨텍스트로 활용하여 질문에 대한 답변을 제공해주세요. 
        * 컨텍스트를 최대한 활용하여 풍부하고 상세히 답변해주세요. 
        * 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해주세요.
        * 검색된 참조링크를 바탕으로 사용자가 클릭할 수 있도록 참조 링크를 **Markdown 형식**(`[제목](URL)`)으로 포함해주세요.
        * 출력언어 설정에 따라 답변해주세요. 

    """

    USER_PROMPT_TEMPLATE = """    
        오늘 날짜는 {current_date}이며, bing search tool에서 검색된 최근 결과를 중심으로 사용자 질문에 답변해주세요. 
    
        사용자 질문:
        {query}
        출력언어:
        {locale}

    """

    USER_PROMPT = PromptTemplate(
        template=USER_PROMPT_TEMPLATE,
        input_variables=["query","current_date"],
        optional_variables=["locale"]

    )

    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,  # Model deployment name
        name="VanilaBingGroundingAgent",  # Name of the agent
        instructions=SYSTEM_PROMPT,  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    print(f"Created agent, ID: {agent.id}")
    
    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    
    print(f"# User: '{USER_INPUT}'")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,  # Role of the message sender
        content=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    locale="ko-KR",
                )
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    message = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    
    print(f"Role: {message.role}")
    # Extract the text content from the message
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text':
                display(Markdown(content_item['text']['value']))
    else:
        display(Markdown(message.content))

    # 7. Show annotations
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text' and 'annotations' in content_item.get('text', {}):
                annotations = content_item['text']['annotations']
                for annotation in annotations:
                    if annotation.get('type') == 'url_citation':
                        url_citation = annotation.get('url_citation', {})
                        print(
                            f"Annotation :> {url_citation.get('url', '')}, source={annotation.get('text', '')}, with "
                            f"start_index={annotation.get('start_index', '')} and end_index={annotation.get('end_index', '')}"
                        )

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, ID: asst_fW1QmPCcTB6YD8vmHyvEaQob
Created thread, ID: thread_WRkJuvBS6Kp4CWVOAdTbwcZe
# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Created message, ID: msg_HTS0MVpPxqwzMnfr9Oh5RIIC
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT


최근 서울 도심에 새로 오픈한 핫플레이스로는 **"리버뷰 197"**라는 한강뷰 루프탑 카페가 있습니다. 이 카페는 한강의 멋진 야경을 감상할 수 있는 곳으로, 특히 인스타그램에서 아주 인기가 높습니다. 카페는 최근 리뉴얼되어 더욱 트렌디한 분위기로 재탄생하였답니다.🌃✨

또한, **홍대 미디어 아트 갤러리**도 주목받고 있습니다. 이곳은 다양한 감각적인 아트 전시와 공연이 열리는 공간으로, 홍대의 젊은 예술 문화와 잘 어우러져 있습니다. 친구들이 추천하는 장소로 많은 사랑을 받고 있죠【3:0†source】【3:4†source】.

추가로, **여의도 한강공원 루프탑 카페**도 인기 있는 핫플레이스로 주목받고 있으며, 신선한 해산물을 즐길 수 있는 **노량진 수산시장**도 현대화되어 관광객들에게 좋은 반응을 얻고 있습니다【3:1†source】. 

새로운 핫플레이스를 찾아 서울을 돌아다닐 준비가 되셨나요? 다양한 문화 경험과 멋진 경치를 즐길 수 있는 기회가 많이 있으니 방문해 보시길 추천드립니다!😊

Annotation :> https://kkyoungii.tistory.com/entry/%E2%80%9C%EC%9A%94%EC%A6%98-%EC%82%AC%EB%9E%8C%EB%93%A4-%EC%97%AC%EA%B8%B0-%EB%8B%A4-%EA%B0%84%EB%8B%A4-2025-%EC%84%9C%EC%9A%B8-%ED%95%AB%ED%94%8C-%EC%99%84%EC%A0%84%EC%A0%95%EB%B3%B5%E2%80%9D, source=【3:0†source】, with start_index=274 and end_index=286
Annotation :> https://derbypro1222.tistory.com/entry/2025%EB%85%84-%EC%84%9C%EC%9A%B8-%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-BEST-10-%EC%9D%B8%EC%8A%A4%ED%83%80-%EA%B0%90%EC%84%B1-%EC%97%AC%ED%96%89%EC%A7%80, source=【3:4†source】, with start_index=286 and end_index=298
Annotation :> https://peppercorntc.com/2025%EB%85%84-%EC%84%9C%EC%9A%B8%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-10%EA%B3%B3-%EB%86%93%EC%B9%98%EB%A9%B4-%ED%9B%84%ED%9A%8C/, source=【3:1†source】, with start_index=408 and end_index=420
Deleted agent


## 🧪 Case 5. BingGrounding (Web Search Only) and Scraping (Vanila)
---

In [28]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from IPython.display import Markdown, display

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")

BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
bing_max_result = int(os.getenv("BING_MAX_RESULT", "10"))

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=BING_GROUNDING_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=bing_max_result)

with project_client:
    

    SYSTEM_PROMPT = """    
        You are an intelligent chatbot that can perform real-time web searches.
    """

    USER_PROMPT_TEMPLATE = """    
        Search the web for: {query} only the top {max_results} most relevant results as a list.
        Don't provide any response except annotations. Always return the annotations from the search results. 
        Today is {current_date}. Searching should be based on the recent information available.

    """

    USER_PROMPT = PromptTemplate(
        template=USER_PROMPT_TEMPLATE,
        input_variables=["query","current_date","max_results"]
    )

    # Create an agent with the Bing Grounding tool
    # agent = project_client.agents.create_agent(
    #     model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,  # Model deployment name
    #     name="GroundingWebSearchAgent",  # Name of the agent
    #     instructions=SYSTEM_PROMPT,  # Instructions for the agent
    #     tools=bing.definitions,  # Attach the Bing Grounding tool
    # )
    # print(f"Created agent, ID: {agent.id}")
    
    agent = project_client.agents.get_agent(agent_id="asst_NzVkvp8sriZHRLWe0sEg3dT5")
    
    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    
    print(f"# User: '{USER_INPUT}'")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,  # Role of the message sender
        content=USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date,
                    max_results=BING_GROUNDING_MAX_RESULTS
                )
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    message = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    
    print(f"Role: {message.role}")
    # Extract the text content from the message
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text':
                display(Markdown(content_item['text']['value']))
    else:
        display(Markdown(message.content))

    # 7. Show annotations
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text' and 'annotations' in content_item.get('text', {}):
                annotations = content_item['text']['annotations']
                for annotation in annotations:
                    if annotation.get('type') == 'url_citation':
                        url_citation = annotation.get('url_citation', {})
                        print(
                            f"Annotation :> {url_citation.get('url', '')}, source={annotation.get('text', '')}, with "
                            f"start_index={annotation.get('start_index', '')} and end_index={annotation.get('end_index', '')}"
                        )

    # Delete the agent when done
    #project_client.agents.delete_agent(agent.id)
    #print("Deleted agent")

Created thread, ID: thread_rNPeL4G0nTuvmiPwBWj4TFO2
# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Created message, ID: msg_g2CVkulHTQjNqKxOos8T4aUf
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT


1. 2025년 서울 핫플레이스 TOP 5 소개, 성수동 카페 거리와 잠실 석촌호수 뷰 카페 등 다양한 인기 명소 소개【3:1†source】.
2. 최근 핫하게 떠오른 신상 카페, 팝업스토어 및 전시회 등 서울의 트렌디한 공간들 소개【3:2†source】.
3. 서울에서 현재 가장 인기 있는 10곳을 소개하며 익선동 한옥거리와 성수동 카페 거리 등 다양한 장소를 언급【3:5†source】.

Annotation :> https://presentlive.tistory.com/entry/%F0%9F%93%8C-2025%EB%85%84-%EC%84%9C%EC%9A%B8-%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-TOP-5-%E2%80%93-%EC%9A%94%EC%A6%98-%EA%B0%80%EC%9E%A5-%ED%95%AB%ED%95%9C-%EA%B3%B3, source=【3:1†source】, with start_index=66 and end_index=78
Annotation :> https://view1976.tistory.com/147, source=【3:2†source】, with start_index=130 and end_index=142
Annotation :> https://pipinena.tistory.com/entry/%EC%84%9C%EC%9A%B8-%EC%A0%8A%EC%9D%80%EC%9D%B4%EB%93%A4%EC%9D%98-%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-TOP-10-%E2%80%93-%EC%9A%94%EC%A6%98-%EB%9C%A8%EB%8A%94-%EC%84%9C%EC%9A%B8-%EC%97%AC%ED%96%89-%EC%BD%94%EC%8A%A4-%EC%B4%9D%EC%A0%95%EB%A6%AC, source=【3:5†source】, with start_index=206 and end_index=218
